# How to Access TESS Data: `astroquery`

## Introduction

This notebook uses the MAST Portal's advanced search options to retrieve
1. all TESS light curves for a single target, and 
2. all data products from an example TESS Guest Investigator program. 

The notbook will show how to do an advanced query with `astroquery.mast` and filter out data products that are not desired, and then download the files of interest from the AWS S3 bucket.




## Imports
- The `Observations` module from `astroquery.mast` is needed to make the query and download the data

NOTE -- If you are running this notebook on the TIKE, as recommended, you should not need to install or update your `astroquery` package

In [1]:
from astroquery.mast import Observations

All data from the TESS Mission are available for free on Amazon Web Services in public S3 buckets. (See the small caveat to this at the end of this notebook.) But we need to tell `astroquery` to download any data directly from the S3 bucket rather than retrieve it from the MAST servers.

NOTE -- Using AWS resources to access public MAST data does **not** require an AWS account for all AWS regions. See the [`astroquery.mast` documentation](https://astroquery.readthedocs.io/en/latest/mast/mast.html#cloud-data-access) for more details.

In [2]:
Observations.enable_cloud_dataset(provider='AWS')

INFO: Using the S3 STScI public dataset [astroquery.mast.cloud]


## Query for a specific target
In this example, we want to retrieve all TESS data (light curves, target pixel files, and data validation files) for the target TIC 7854182, a known $\delta$ Scuti star in an eclipsing binary (Chen et al. 2022, Kahraman Aliçavuş et al. 2017, Liakos & Niarchos 2017).

Feel free to use your favorite target here instead!

In [3]:
target_name = "TIC 7854182"

In [4]:
obs = Observations.query_object(target_name, radius="0s")


In [5]:
print(f"TOTAL Number of Observations available for {target_name}: \n{len(obs)}")

TOTAL Number of Observations available for TIC 7854182: 
47


The total number of Observations include all Missions (HST, JWST, etc.) in the MAST collection, as we have not filtered down to just TESS data yet.

Next, we want to apply the advanced search filters to narrow our results, and retrieve the list of data products that are associated with each Observation.

The filter names that can be used in the `get_product_list` function are listed on the [MAST API Field Description Page](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html).

To keep this search simple, we will use the following filters:
- obs_collection to specify that we only want TESS data
- dataproduct_type to specify that we want timeseries data

In [6]:
filters = (obs['obs_collection'] == "TESS") & (obs['dataproduct_type'] == 'timeseries')

# Pick which products we want to retrieve using the Advanced Search options
data_prod = Observations.get_product_list(obs[filters])


In [7]:
print(f"Number of TESS time series data products available for {target_name}: \n{len(data_prod)}")

Number of TESS time series data products available for TIC 7854182: 
76


In [8]:
display(data_prod[-5:])

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str8,str4,str10,str47,str33,str1,str81,str7,str28,str3,str1,str4,str20,str7,str63,int64,str8,str6,int64
91541675,TESS,timeseries,tess2022190063128-s0054-0000000007854182-0227-s,TCE summary report,S,mast:TESS/product/tess2022190092448-s0054-s0054-0000000007854182-01-00645_dvs.pdf,INFO,Minimum Recommended Products,DVS,--,SPOC,42dc8a2f6f,G04123,tess2022190092448-s0054-s0054-0000000007854182-01-00645_dvs.pdf,1261215,91541675,PUBLIC,3
91541675,TESS,timeseries,tess2022190063128-s0054-0000000007854182-0227-s,TCE summary report,S,mast:TESS/product/tess2022190092448-s0054-s0054-0000000007854182-02-00645_dvs.pdf,INFO,Minimum Recommended Products,DVS,--,SPOC,42dc8a2f6f,G04123,tess2022190092448-s0054-s0054-0000000007854182-02-00645_dvs.pdf,1235945,91541675,PUBLIC,3
91541675,TESS,timeseries,tess2022190063128-s0054-0000000007854182-0227-s,Data validation time series,S,mast:TESS/product/tess2022190092448-s0054-s0054-0000000007854182-00645_dvt.fits,SCIENCE,Minimum Recommended Products,DVT,--,SPOC,42dc8a2f6f,G04123,tess2022190092448-s0054-s0054-0000000007854182-00645_dvt.fits,4651200,91541675,PUBLIC,3
91541675,TESS,timeseries,tess2022190063128-s0054-0000000007854182-0227-s,Light curves,S,mast:TESS/product/tess2022190063128-s0054-0000000007854182-0227-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,42dc8a2f6f,G04123,tess2022190063128-s0054-0000000007854182-0227-s_lc.fits,1918080,91541675,PUBLIC,3
91541675,TESS,timeseries,tess2022190063128-s0054-0000000007854182-0227-s,Target pixel files,S,mast:TESS/product/tess2022190063128-s0054-0000000007854182-0227-s_tp.fits,SCIENCE,Minimum Recommended Products,TP,--,SPOC,spoc-5.0.74-20220814,G04123,tess2022190063128-s0054-0000000007854182-0227-s_tp.fits,46281600,91541675,PUBLIC,2


If we decide that we only want to download, e.g., the light curve files, we can further filter the products list based on the columns/fields in the table.

Descriptions of each of the product filters that can be used in the `filter_products` function are available on the [MAST API Product Field Description page](https://mast.stsci.edu/api/v0/_productsfields.html).

In [9]:
data_prod.colnames


['obsID',
 'obs_collection',
 'dataproduct_type',
 'obs_id',
 'description',
 'type',
 'dataURI',
 'productType',
 'productGroupDescription',
 'productSubGroupDescription',
 'productDocumentationURL',
 'project',
 'prvversion',
 'proposal_id',
 'productFilename',
 'size',
 'parent_obsid',
 'dataRights',
 'calib_level']

In [10]:
# Select which files to download from the S3 bucket by applying additional filters
filt_prod = Observations.filter_products(data_prod, description='Light curves')


In [11]:
print(f"Number of TESS light curves available for {target_name}: \n{len(filt_prod)}")

Number of TESS light curves available for TIC 7854182: 
7


In [12]:
display(filt_prod)

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str8,str4,str10,str47,str33,str1,str81,str7,str28,str3,str1,str4,str20,str7,str63,int64,str8,str6,int64
27626482,TESS,timeseries,tess2020133194932-s0025-0000000007854182-0182-s,Light curves,S,mast:TESS/product/tess2020133194932-s0025-0000000007854182-0182-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,b9bb72fe01,G022062,tess2020133194932-s0025-0000000007854182-0182-s_lc.fits,1877760,27626482,PUBLIC,3
27671902,TESS,timeseries,tess2020160202036-s0026-0000000007854182-0188-s,Light curves,S,mast:TESS/product/tess2020160202036-s0026-0000000007854182-0188-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,2f78f64b59,G022062,tess2020160202036-s0026-0000000007854182-0188-s_lc.fits,1820160,27671902,PUBLIC,3
62346553,TESS,timeseries,tess2021175071901-s0040-0000000007854182-0211-s,Light curves,S,mast:TESS/product/tess2021175071901-s0040-0000000007854182-0211-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,0f075725c1,G04123,tess2021175071901-s0040-0000000007854182-0211-s_lc.fits,2062080,62346553,PUBLIC,3
62789558,TESS,timeseries,tess2021204101404-s0041-0000000007854182-0212-s,Light curves,S,mast:TESS/product/tess2021204101404-s0041-0000000007854182-0212-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,c5eccc120c,G04123,tess2021204101404-s0041-0000000007854182-0212-s_lc.fits,1944000,62789558,PUBLIC,3
87637038,TESS,timeseries,tess2022138205153-s0052-0000000007854182-0224-s,Light curves,S,mast:TESS/product/tess2022138205153-s0052-0000000007854182-0224-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,36a1b8e3de,G04123,tess2022138205153-s0052-0000000007854182-0224-s_lc.fits,1791360,87637038,PUBLIC,3
87743465,TESS,timeseries,tess2022164095748-s0053-0000000007854182-0226-s,Light curves,S,mast:TESS/product/tess2022164095748-s0053-0000000007854182-0226-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,36a1b8e3de,G04123,tess2022164095748-s0053-0000000007854182-0226-s_lc.fits,1828800,87743465,PUBLIC,3
91541675,TESS,timeseries,tess2022190063128-s0054-0000000007854182-0227-s,Light curves,S,mast:TESS/product/tess2022190063128-s0054-0000000007854182-0227-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,42dc8a2f6f,G04123,tess2022190063128-s0054-0000000007854182-0227-s_lc.fits,1918080,91541675,PUBLIC,3


Now, we're ready to download the data from the AWS S3 bucket!

When set to True, the `cloud_only` parameter in `download_products` skips any data products that are not available in the cloud; all TESS Mission data are available through AWS, so none of the selected data products should be skipped.

NOTE -- If you try to download the same file(s) more than once (e.g., by running the cell below multiple times), you should get the message "Found cached file" instead of "Downloading URL" in the printed manifest.

In [13]:
manifest = Observations.download_products(filt_prod, cloud_only=True)


All TESS light curves for TIC 7854182 have been downloaded, and we're ready to start our science!

## Query for a specific Guest Investigator program
In this example, we want to retrieve all TESS data (light curves, target pixel files, and data validation files) associated with Guest Investigator program G05101 from Cycle 5 (PI: Susan Mullally).

Feel free to use any program here! The list of program IDs can be found at the [TESS GI List of Approved Programs](https://heasarc.gsfc.nasa.gov/docs/tess/approved-programs.html).

In [14]:
pid = "G05101"

In [15]:
obs_pid = Observations.query_criteria(obs_collection = "TESS",
                                  proposal_id = f"*{pid}*")

In [16]:
print(f"TOTAL Number of Observations available for {pid}: \n{len(obs_pid)}")

TOTAL Number of Observations available for G05101: 
49


In [17]:
display(obs_pid[:5])

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str4,str4,str10,str4,str4,str7,str9,str1,str52,float64,float64,str10,str14,int64,float64,float64,float64,float64,float64,str1,float64,str34,str1,int64,str43,str1,str79,str6,bool,float64,str9,str9
science,TESS,SPOC,Photometer,TESS,TESS,Optical,298411553,--,tess2022330142927-s0059-0000000298411553-0248-s,149.476234232293,85.4946883170451,timeseries,"Ricker, George",3,59909.7650365625,59936.18904974537,120.0,600.0,1000.0,--,59962.0,G05101_G05084_G05115_G05081,--,59,CIRCLE 149.47623423 85.49468832 0.00138889,--,mast:TESS/product/tess2022330142927-s0059-0000000298411553-0248-s_lc.fits,PUBLIC,False,nan,113392111,210500653
science,TESS,SPOC,Photometer,TESS,TESS,Optical,219097989,--,tess2022357055054-s0060-0000000219097989-0249-s,269.658310781969,66.7811423559296,timeseries,"Ricker, George",3,59936.40188498842,59962.08243238426,120.0,600.0,1000.0,--,59982.0,G05101,--,60,CIRCLE 269.65831078 66.78114236 0.00138889,--,mast:TESS/product/tess2022357055054-s0060-0000000219097989-0249-s_lc.fits,PUBLIC,False,nan,114309714,211965772
science,TESS,SPOC,Photometer,TESS,TESS,Optical,383623835,--,tess2022330142927-s0059-0000000383623835-0248-s,270.628087535846,58.6272659486741,timeseries,"Ricker, George",3,59909.76229211806,59936.1858008912,120.0,600.0,1000.0,--,59962.0,G05101,--,59,CIRCLE 270.62808754 58.62726595 0.00138889,--,mast:TESS/product/tess2022330142927-s0059-0000000383623835-0248-s_lc.fits,PUBLIC,False,nan,113395560,210506880
science,TESS,SPOC,Photometer,TESS,TESS,Optical,5914520,--,tess2023096110322-s0064-0000000005914520-0257-s,193.312720118104,-18.5222248428656,timeseries,"Ricker, George",3,60040.63527681713,60067.43903234954,120.0,600.0,1000.0,--,60087.0,G05101,--,64,CIRCLE 193.31272012 -18.52222484 0.00138889,--,mast:TESS/product/tess2023096110322-s0064-0000000005914520-0257-s_lc.fits,PUBLIC,False,nan,139084486,254133156
science,TESS,SPOC,Photometer,TESS,TESS,Optical,420814525,--,tess2022244194134-s0056-0000000420814525-0243-s,330.794887332661,18.8843189579296,timeseries,"Ricker, George",3,59824.761134409724,59852.642480821756,120.0,600.0,1000.0,--,59873.0,G05101_G05015_G05144_G05069,--,56,CIRCLE 330.79488733 18.88431896 0.00138889,--,mast:TESS/product/tess2022244194134-s0056-0000000420814525-0243-s_lc.fits,PUBLIC,False,nan,97942919,184143980


Next, we will retrieve the list of data products that are associated with each Observation.


In [18]:
# Pick which products we want to retrieve
data_prod_pid = Observations.get_product_list(obs_pid)

In [19]:
print(f"Number of TESS data products available for {pid}: \n{len(data_prod_pid)}")

Number of TESS data products available for G05101: 
106


In [20]:
display(data_prod[10:17])

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str8,str4,str10,str47,str33,str1,str81,str7,str28,str3,str1,str4,str20,str7,str63,int64,str8,str6,int64
64265093,TESS,timeseries,tess2019199201929-s0014-s0041-0000000007854182,TCE summary report,S,mast:TESS/product/tess2019199201929-s0014-s0041-0000000007854182-02-00512_dvs.pdf,INFO,Minimum Recommended Products,DVS,--,SPOC,c5eccc120c,G04123,tess2019199201929-s0014-s0041-0000000007854182-02-00512_dvs.pdf,1806484,64265093,PUBLIC,3
64265093,TESS,timeseries,tess2019199201929-s0014-s0041-0000000007854182,TCE summary report,S,mast:TESS/product/tess2019199201929-s0014-s0041-0000000007854182-03-00512_dvs.pdf,INFO,Minimum Recommended Products,DVS,--,SPOC,c5eccc120c,G04123,tess2019199201929-s0014-s0041-0000000007854182-03-00512_dvs.pdf,1520264,64265093,PUBLIC,3
64265093,TESS,timeseries,tess2019199201929-s0014-s0041-0000000007854182,Data validation time series,S,mast:TESS/product/tess2019199201929-s0014-s0041-0000000007854182-00512_dvt.fits,SCIENCE,Minimum Recommended Products,DVT,--,SPOC,c5eccc120c,G04123,tess2019199201929-s0014-s0041-0000000007854182-00512_dvt.fits,96050880,64265093,PUBLIC,3
96861135,TESS,timeseries,tess2019199201929-s0014-s0055-0000000007854182,full data validation report,S,mast:TESS/product/tess2019199201929-s0014-s0055-0000000007854182-00652_dvr.pdf,INFO,--,DVR,--,SPOC,a3d64e14a5,N/A,tess2019199201929-s0014-s0055-0000000007854182-00652_dvr.pdf,62597369,96861135,PUBLIC,3
96861135,TESS,timeseries,tess2019199201929-s0014-s0055-0000000007854182,full data validation report (xml),S,mast:TESS/product/tess2019199201929-s0014-s0055-0000000007854182-00652_dvr.xml,INFO,--,DVR,--,SPOC,a3d64e14a5,N/A,tess2019199201929-s0014-s0055-0000000007854182-00652_dvr.xml,1777094,96861135,PUBLIC,3
96861135,TESS,timeseries,tess2019199201929-s0014-s0055-0000000007854182,Data validation mini report,S,mast:TESS/product/tess2019199201929-s0014-s0055-0000000007854182-00652_dvm.pdf,INFO,Minimum Recommended Products,DVM,--,SPOC,a3d64e14a5,N/A,tess2019199201929-s0014-s0055-0000000007854182-00652_dvm.pdf,21701976,96861135,PUBLIC,3
96861135,TESS,timeseries,tess2019199201929-s0014-s0055-0000000007854182,TCE summary report,S,mast:TESS/product/tess2019199201929-s0014-s0055-0000000007854182-01-00652_dvs.pdf,INFO,Minimum Recommended Products,DVS,--,SPOC,a3d64e14a5,N/A,tess2019199201929-s0014-s0055-0000000007854182-01-00652_dvs.pdf,1366512,96861135,PUBLIC,3


As above, if we decide that we only want to download, e.g., the light curve files, we can further filter the products list based on the columns/fields in the table.

For our purposes, this step is optional; I'm going to download **all** of the available data products for this program ID.

In [ ]:
# OPTIONAL
# Select which files to download from the S3 bucket by applying additional filters
# filt_prod = Observations.filter_products(data_prod_pid, description='Light curves')

Now, we're ready to download the data from the AWS S3 bucket!

By default, all downloads will be placed in a `./mastDownload/` directory on the TIKE. If you'd like to change this directory, use the `download_dir` parameter in the `download_products` functions. For example, you may want to place all products downloaded for GI program G05101 in a directory named `./G05101`, as shown below.


In [21]:
manifest = Observations.download_products(data_prod_pid, download_dir=f'{pid}', cloud_only=True)


ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


If you scroll through the above manifest, you may notice an error:
```
ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]
WARNING: Skipping file... [astroquery.mast.observations]
```

So, what happened here? Let's check the manifest to see which files were not downloaded.


In [22]:
display(manifest[manifest['Status']!='COMPLETE'])

Local Path,Status,Message,URL
str138,str8,object,object
G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-00810_dvr.pdf,SKIPPED,None,None
G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-00810_dvm.pdf,SKIPPED,None,None
G05101/mastDownload/TESS/tess2022244194134-s0056-0000000420814525-0243-s/tess2022245180045-s0056-s0056-0000000420814525-00661_dvr.pdf,SKIPPED,None,None
G05101/mastDownload/TESS/tess2022244194134-s0056-0000000420814525-0243-s/tess2022245180045-s0056-s0056-0000000420814525-00661_dvm.pdf,SKIPPED,None,None


When we say that "ALL" data from the TESS Mission are available in AWS S3 buckets, there is a small caveat.

The TESS Full Data Validation Reports (`*_dvr.pdf`) and Mini Data Validation Reports (`*_drm.pdf`), which are produced for all TCEs associated with a particular host star, are **not currently** hosted on AWS.

As MAST moves towards a more cloud-based model for data access, MAST is considering adding these types of data products to the AWS S3 bucket, but for now, if we want to download them to our local directory, we'll need to download them directly from MAST. To do this, we set `cloud_only=False`. If set to False and cloud data access is enabled (`enable_cloud_dataset` above), files that are not found in the cloud will be downloaded from MAST.

In [23]:
manifest = Observations.download_products(data_prod_pid, download_dir=f'{pid}', cloud_only=False)


ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


 [Done]
INFO: Found cached file G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-00810_dvr.xml with expected size 374231. [astroquery.mast.cloud]

ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


 [Done]
INFO: Found cached file G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-01-00810_dvs.pdf with expected size 703837. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2018206190142-s0001-s0069-0000000200779640/tess2018206190142-s0001-s0069-0000000200779640-00810_dvt.fits with expected size 53426880. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000207440438-0243-s/tess2022244194134-s0056-0000000207440438-0243-s_lc.fits with expected size 2039040. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000207440438-0243-s/tess2022244194134-s0056-0000000207440438-0243-s_tp.fits with expected size 49193280. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000219089401-0243-s/tess2022244194134-s0056-0000000219089401-0243-s_lc.fits with e

ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


 [Done]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000420814525-0243-s/tess2022245180045-s0056-s0056-0000000420814525-00661_dvr.xml with expected size 130901. [astroquery.mast.cloud]


ERROR: Error pulling from S3 bucket: 'productFilename' [astroquery.mast.observations]


INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000420814525-0243-s/tess2022245180045-s0056-s0056-0000000420814525-01-00661_dvs.pdf with expected size 1026134. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000420814525-0243-s/tess2022245180045-s0056-s0056-0000000420814525-00661_dvt.fits with expected size 4046400. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000420814525-0243-s/tess2022244194134-s0056-0000000420814525-0243-s_lc.fits with expected size 2039040. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000420814525-0243-s/tess2022244194134-s0056-0000000420814525-0243-s_tp.fits with expected size 58026240. [astroquery.mast.cloud]
INFO: Found cached file G05101/mastDownload/TESS/tess2022244194134-s0056-0000000452740376-0243-a_fast/tess2022244194134-s0056-0000000452740376-0243-a_fast-lc.fits wi

All TESS data products (light curves, target pixel files, and data validation files) associated with Guest Investigator program G05101 have been downloaded

## About this Notebook

**Author:** Hannah M. Lewis, STScI Data Scientist

**Updated On:** 2023-01-05, 2023-12-19

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="STScI logo" width="200px"/>